<a href="https://colab.research.google.com/github/OndraDol/Nicotine-pouches-tracker/blob/main/notebooks/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [14]:
# --- ČÁST 1: NASTAVENÍ PROSTŘEDÍ A INSTALACE COMFYUI ---
print("########## ČÁST 1: NASTAVENÍ PROSTŘEDÍ A INSTALACE COMFYUI ##########")
import os # Import modulu os hned na začátku

# @title Environment Setup - Základní nastavení
USE_GOOGLE_DRIVE = True
UPDATE_COMFY_UI = True
COMFYUI_REPO_URL = "https://github.com/comfyanonymous/ComfyUI.git"

# Nastavení WORKSPACE a připojení Google Drive
if USE_GOOGLE_DRIVE:
    print("Připojuji Google Drive...")
    %cd /
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COMFYUI_PATH = "/content/drive/MyDrive/ComfyUI"
    print(f"COMFYUI_PATH (cesta na Google Drive) nastavena na: {COMFYUI_PATH}")
    !mkdir -p /content/drive/MyDrive # Zajistí existenci MyDrive
    %cd /content/drive/MyDrive
else:
    COMFYUI_PATH = "/content/ComfyUI"
    print(f"COMFYUI_PATH (lokální cesta v Colabu) nastavena na: {COMFYUI_PATH}")
    %cd /content

print(f"Kontroluji existenci adresáře ComfyUI v: {COMFYUI_PATH}")
if not os.path.exists(COMFYUI_PATH): # Použití os.path.exists pro kontrolu
    print(f"-= Prvotní nastavení ComfyUI: Klonuji repozitář {COMFYUI_REPO_URL} do {COMFYUI_PATH} =-")
    !git clone {COMFYUI_REPO_URL} "{COMFYUI_PATH}" # Klonování přímo do cílového adresáře
else:
    print(f"-= Adresář {COMFYUI_PATH} již existuje. =-")

%cd $COMFYUI_PATH
print(f"Aktuální pracovní adresář: ")
!pwd

if UPDATE_COMFY_UI:
    print("-= Aktualizuji ComfyUI =-")
    !git pull
else:
    print("Aktualizace ComfyUI je přeskočena.")

print("-= Instaluji závislosti pro ComfyUI =-")
# Zvýšení timeoutu pro pip a vypnutí progress baru, pokud by byly problémy
!pip config set global.progress_bar off
!pip install --timeout=900 xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117
!pip config unset global.progress_bar
print("Instalace závislostí ComfyUI dokončena.")

print("\nKontrola cest po nastavení ComfyUI:")
if USE_GOOGLE_DRIVE:
    print("Obsah /content/drive:")
    !ls /content/drive || echo "CHYBA: Adresář /content/drive nenalezen."
    print(f"\nObsah /content/drive/MyDrive:")
    !ls /content/drive/MyDrive || echo "CHYBA: Adresář /content/drive/MyDrive nenalezen."
print(f"\nObsah {COMFYUI_PATH}:")
!ls -l "{COMFYUI_PATH}" || echo "CHYBA: Adresář {COMFYUI_PATH} nenalezen."
print(f"\nObsah {COMFYUI_PATH}/custom_nodes (může být zatím prázdný nebo obsahovat jen příklady):")
!ls -l "{COMFYUI_PATH}/custom_nodes" || echo "CHYBA: Adresář {COMFYUI_PATH}/custom_nodes nenalezen (bude vytvořen v další části)."
print("--- KONEC: ČÁST 1 ---")

# --- ČÁST 2: STAHOVÁNÍ MODELŮ ---
print("\n\n########## ČÁST 2: STAHOVÁNÍ MODELŮ ##########")
# Ujistíme se, že jsme v kořenovém adresáři ComfyUI pro relativní cesty modelů
%cd $COMFYUI_PATH
print(f"Aktuální pracovní adresář pro stahování modelů: ")
!pwd

# Vytvoření základních adresářů pro modely, pokud neexistují
print("Vytvářím potřebné adresáře pro modely (pokud již neexistují)...")
!mkdir -p ./models/checkpoints
!mkdir -p ./models/clip_vision
!mkdir -p ./models/vae
!mkdir -p ./models/loras
!mkdir -p ./models/ipadapter
!mkdir -p ./models/controlnet
!mkdir -p ./models/upscale_models

print("\nZahajuji stahování modelů (může to trvat déle)...")

# Checkpoints SDXL
print("\nStahuji SDXL Checkpointy...")
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -O ./models/checkpoints/sd_xl_base_1.0.safetensors
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -O ./models/checkpoints/sd_xl_refiner_1.0.safetensors

# SDXL ReVision CLIP Vision
print("\nStahuji SDXL ReVision (CLIP Vision)...")
!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -O ./models/clip_vision/clip_vision_g.safetensors

# SD1.5 Checkpoint
print("\nStahuji SD1.5 Checkpoint...")
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -O ./models/checkpoints/v1-5-pruned-emaonly-fp16.safetensors

# Doporučený fotorealistický model pro SD1.5 (zakomentováno, odkomentuj pokud chceš)
# print("\nStahuji (volitelně) Realistic Vision V6.0 B1...")
# !wget -c https://civitai.com/api/download/models/130072 -O ./models/checkpoints/realisticVisionV60B1_v60B1VAE.safetensors

# VAE
print("\nStahuji VAE...")
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -O ./models/vae/vae-ft-mse-840000-ema-pruned.safetensors

# Loras
print("\nStahuji LoRA modely...")
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -O ./models/loras/sd_xl_offset_example-lora_1.0.safetensors

# ICEdit LoRA - problém se stahováním (401 Unauthorized), prozatím zakomentováno
# print("\nPOZNÁMKA: Pokus o stažení ICEdit LoRA (https://huggingface.co/RiverLeaf/ICEdit/resolve/main/icedit.safetensors) byl vynechán,")
# print("protože původní odkaz vracel chybu 401 Unauthorized. Model nemusí být na této adrese dostupný.")
# print("Pro 'instruction-based editing' zvažte použití ControlNet modelu 'control_v11e_sd15_ip2p_fp16.safetensors'.")
# # !wget -c https://huggingface.co/RiverLeaf/ICEdit/resolve/main/icedit.safetensors -P ./models/loras/ # Tento řádek je zakomentován

!wget -c https://civitai.com/api/download/models/122359 -O ./models/loras/addMoreDetailsDetailEnhancerTweakerLora_v10.safetensors

# IP-Adapter modely
print("\nStahuji IP-Adapter modely...")
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O ./models/clip_vision/ip_adapter_sd15_image_encoder.safetensors # ViT-H image encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P ./models/ipadapter/
# Pro SDXL IP-Adapter se obvykle používá ViT-G image encoder (clip_vision_g.safetensors), který byl stažen výše.

# ControlNet modely pro SD1.5
print("\nStahuji ControlNet modely pro SD1.5...")
CONTROLNET_MODELS_SD15 = [
    "control_v11e_sd15_ip2p_fp16.safetensors", "control_v11e_sd15_shuffle_fp16.safetensors",
    "control_v11p_sd15_canny_fp16.safetensors", "control_v11f1p_sd15_depth_fp16.safetensors",
    "control_v11p_sd15_inpaint_fp16.safetensors", "control_v11p_sd15_lineart_fp16.safetensors",
    "control_v11p_sd15_mlsd_fp16.safetensors", "control_v11p_sd15_normalbae_fp16.safetensors",
    "control_v11p_sd15_openpose_fp16.safetensors", "control_v11p_sd15_scribble_fp16.safetensors",
    "control_v11p_sd15_seg_fp16.safetensors", "control_v11p_sd15_softedge_fp16.safetensors",
    "control_v11u_sd15_tile_fp16.safetensors"
]
for model_name in CONTROLNET_MODELS_SD15:
    print(f"Stahuji {model_name}...")
    !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/{model_name} -P ./models/controlnet/

# ControlNet LoRA modely pro SDXL
print("\nStahuji ControlNet LoRA modely pro SDXL...")
CONTROLNET_LORA_MODELS_SDXL = [
    "control-lora-canny-rank256.safetensors", "control-lora-depth-rank256.safetensors",
    "control-lora-recolor-rank256.safetensors", "control-lora-sketch-rank256.safetensors"
]
for model_name in CONTROLNET_LORA_MODELS_SDXL:
    print(f"Stahuji {model_name}...")
    !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/{model_name} -P ./models/controlnet/

# ESRGAN upscale modely
print("\nStahuji ESRGAN upscale modely...")
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/
print("Stahování modelů dokončeno.")
print("--- KONEC: ČÁST 2 ---")


# --- ČÁST 3: INSTALACE CONTROLNET PREPROCESSORS ---
# --- ČÁST 3: INSTALACE CONTROLNET PREPROCESSORS (Manuální ZIP varianta) ---
print("\n\n########## ČÁST 3: INSTALACE CONTROLNET PREPROCESSORS (Manuální ZIP varianta) ##########")
# Ujistíme se, že jsme v COMFYUI_PATH
%cd $COMFYUI_PATH
print(f"Aktuální pracovní adresář: ")
!pwd

# Vytvoříme adresář custom_nodes, pokud by náhodou ještě neexistoval
!mkdir -p ./custom_nodes

# Přechod do custom_nodes
%cd ./custom_nodes
print(f"Aktuální pracovní adresář pro instalaci custom nodů: ")
!pwd

# Definice Python proměnných
zip_url = "https://github.com/Fannovel16/comfy_controlnet_preprocessors/archive/refs/heads/main.zip"
target_dir = "comfy_controlnet_preprocessors" # Název adresáře, jaký chceme mít po rozbalení a přejmenování
zip_filename = "comfy_controlnet_preprocessors_main.zip" # Název staženého zip souboru
unzipped_dirname = "comfy_controlnet_preprocessors-main" # GitHub obvykle přidává "-main" nebo "-master" k názvu složky v ZIPu

print(f"\nStahuji a instaluji '{target_dir}' z '{zip_url}'...")

if not os.path.isdir(target_dir):
    print(f"Adresář '{target_dir}' neexistuje. Zahajuji stahování ZIPu.")
    # Smazání starých souborů, pokud by tam náhodou byly z předchozích neúspěšných pokusů
    !rm -f {zip_filename}
    !rm -rf {unzipped_dirname} # Smaže adresář, pokud by zbyl z předchozího rozbalení
    # Není potřeba mazat target_dir, protože if not os.path.isdir(target_dir) by neprošlo

    print(f"Stahuji {zip_url} jako {zip_filename}...")
    !wget --quiet {zip_url} -O {zip_filename} # Přidáno --quiet pro méně výpisu, -O pro správný název

    if os.path.exists(zip_filename): # Kontrola existence staženého ZIPu
        print("ZIP soubor stažen, rozbaluji...")
        !unzip -q {zip_filename} # -q pro tichý výstup (méně zpráv)
        !rm {zip_filename} # Smazání ZIP souboru po rozbalení

        if os.path.isdir(unzipped_dirname): # Kontrola, zda se vytvořil adresář s očekávaným názvem
            print(f"Přejmenovávám '{unzipped_dirname}' na '{target_dir}'...")
            !mv {unzipped_dirname} {target_dir}
            print(f"Adresář '{target_dir}' úspěšně vytvořen a přejmenován.")
        else:
            print(f"CHYBA: Očekávaný adresář '{unzipped_dirname}' po rozbalení nebyl nalezen. Zkontrolujte název větve (main/master) v ZIP URL.")
    else:
        print(f"CHYBA: Stažení ZIP souboru '{zip_filename}' selhalo.")
else:
    print(f"Adresář '{target_dir}' již existuje. Předpokládám, že je již nainstalován.")
    print("Pokud chcete přeinstalovat/aktualizovat, smažte adresář '{target_dir_cn_preproc}' v custom_nodes a spusťte buňku znovu.")


print(f"\nPokouším se nainstalovat závislosti pro '{target_dir}' (pokud adresář existuje)...")
if os.path.isdir(target_dir):
    print(f"Přecházím do adresáře '{target_dir}'...")
    %cd $target_dir
    print(f"Aktuální adresář před instalací závislostí:")
    !pwd
    print("Spouštím instalaci závislostí (python install.py)...")
    !python install.py
    %cd .. # Návrat z adresáře preprocesorů zpět do custom_nodes
else:
    print(f"CHYBA: Adresář '{target_dir}' nebyl nalezen. Instalace závislostí nemůže pokračovat.")

# Návrat do hlavního adresáře ComfyUI
%cd $COMFYUI_PATH
print(f"\nAktuální pracovní adresář po všech operacích: ")
!pwd
print("--- KONEC: ČÁST 3 (Instalace ControlNet Preprocessors) ---")

# print("\n\n########## ČÁST 4: SPUŠTĚNÍ COMFYUI (PŘÍKLAD) ##########")
# print("Pro spuštění ComfyUI použij příslušnou buňku ve tvém Colab notebooku, NEBO V NOVÉ BUŇCE spusť něco jako:")
# print("#!python main.py --listen --port 8188 --enable-cors --xformers")
# print("#NEBO pro veřejný odkaz (může vyžadovat další nastavení/balíčky jako pyngrok):")
# print("#!python main.py --share")

print("\nPokud vše proběhlo bez chyb, ComfyUI by mělo být připraveno ke spuštění v samostatné buňce.")

########## ČÁST 1: NASTAVENÍ PROSTŘEDÍ A INSTALACE COMFYUI ##########
Připojuji Google Drive...
/
Mounted at /content/drive
COMFYUI_PATH (cesta na Google Drive) nastavena na: /content/drive/MyDrive/ComfyUI
/content/drive/MyDrive
Kontroluji existenci adresáře ComfyUI v: /content/drive/MyDrive/ComfyUI
-= Adresář /content/drive/MyDrive/ComfyUI již existuje. =-
/content/drive/MyDrive/ComfyUI
Aktuální pracovní adresář: 
/content/drive/MyDrive/ComfyUI
-= Aktualizuji ComfyUI =-
Already up to date.
-= Instaluji závislosti pro ComfyUI =-
Writing to /root/.config/pip/pip.conf
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
Writing to /root/.config/pip/pip.conf
Instalace závislostí ComfyUI dokončena.

Kontrola cest po nastavení ComfyUI:
Obsah /content/drive:
MyDrive  Othercomputers

Obsah /content/drive/MyDrive:
 5s.7z			   Hired.gsheet
 AAA			   Inbody.gsheet
'AAA Recruitment.doc'	

### Run ComfyUI with cloudflared (Recommended Way)




### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import threading

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server